In [1]:
import requests
import re
import time
import pandas as pd
from lxml import etree
from urllib.request import urlopen, Request

In [8]:
url = "http://www.twbest1.com/"

def getPage(url):
    res = requests.get(url)
    time.sleep(1)
    res.encoding = 'big5'
    htmltext = res.text
    # htmltext = urlopen(Request(url, headers={'User-Agent': 'Mozilla'})).read().decode('cp950')
    return htmltext
# def getDriver(url):
#     webdriver_path = r'C:/geckodriver.exe'
#     driver = webdriver.Firefox(executable_path=webdriver_path)
#     driver.get(url)
#     return driver

def gethreflist(htmltext):
    xdoc = etree.HTML(htmltext)
    href_list = list(xdoc.xpath('//area//@href'))
    href_list = list(filter(lambda x:re.search(r"(https://www\.twbest1.com)/pc[0-9]*/page[0-9]*\.htm*",x),href_list))
    # href_list = list(filter(lambda x:re.search(r"page172|page100|page280",x),href_list))
    return href_list

# replace html reserve sign to normal sign
def reserve_trans(htmltext):
    htmltext = re.sub(r"\u0020","",htmltext)
    htmltext = re.sub(r"\\n","",htmltext)
    htmltext = re.sub(r"&lt;",r"<",htmltext)
    htmltext = re.sub(r"&nbsp;",r"<",htmltext)
    htmltext = re.sub(r"&gt;",r">",htmltext)
    htmltext = re.sub(r"&amp;",r"&",htmltext)
    htmltext = re.sub(r"&quot;",r'"',htmltext)
    htmltext = re.sub(r"&apos;",r"'",htmltext)
    return htmltext
    
# def parser(htmltext):
#     # xdoc = etree.HTML(htmltext)
#     # site_name = xdoc.xpath("//title")
#     # ◎ xx：</span>
#     data = re.findall(r"\u25ce(.*?)[\uff1a\u003a][^</font>)](.*?)</span>",htmltext) or ["None"]
#     # 月
#     period_list = re.findall(r"(.\u6708)",htmltext) or ["None"]
#     return data

def parser_simple(htmltext):
    xdoc = etree.HTML(htmltext)
    site_name = xdoc.xpath("//title/text()")
    area = re.findall(r"(..\u5e02)(..\u5340)|(..\u7e23)(..\u9109)|(..\u7e23)(..\u93ae)|(..\u7e23)(..\u5e02)",htmltext)
    return list(zip(site_name,area))

In [9]:
url1 = "https://www.twbest1.com/pc001048/page070.htm"
url2 = "https://www.twbest1.com/pc100150/page237.htm"
print(parser_simple(reserve_trans(getPage(url1))))
print(parser_simple(reserve_trans(getPage(url2))))

[('菁桐老街', ('新北市', '平溪區', '', '', '', '', '', ''))]
[('原住民文化主題公園&忘憂湖', ('台北市', '士林區', '', '', '', '', '', ''))]


In [10]:
href_list = gethreflist(getPage(url))
data = [parser_simple(reserve_trans(getPage(href))) for href in href_list]

In [11]:
data

[[('清水斷崖', ('', '', '花蓮縣', '秀林鄉', '', '', '', ''))],
 [('太平山國家森林遊樂區', ('', '', '宜蘭縣', '大同鄉', '', '', '', ''))],
 [('桃園機場捷運(沿線景點)', ('桃園市', '蘆竹區', '', '', '', '', '', ''))],
 [],
 [],
 [],
 [('基隆正濱漁港', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('阿根納造船廠遺址', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('基隆碧砂漁港', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('基隆和平島', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('望幽谷', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [('海門天險', ('基隆市', '中正區', '', '', '', '', '', ''))],
 [],
 [],
 [('宜蘭．梅花湖', ('', '', '宜蘭縣', '冬山鄉', '', '', '', ''))],
 [('宜蘭．福山植物園', ('', '', '宜蘭縣', '員山鄉', '', '', '', ''))],
 [('宜蘭明池', ('', '', '宜蘭縣', '大同鄉', '', '', '', ''))],
 [('宜蘭望龍埤', ('', '', '宜蘭縣', '員山鄉', '', '', '', ''))],
 [('南方澳漁港', ('', '', '', '', '宜蘭縣', '蘇澳鎮', '', ''))],
 [],
 [],
 [],
 [('幾米廣場&丟丟噹森林', ('', '', '', '', '', '', '宜蘭縣', '宜蘭市'))],
 [],
 [('宜蘭‧蘭陽博物館', ('', '', '', '', '宜蘭縣', '頭城鎮', '', ''))],
 [('烏石港', ('', '', '', '', '宜蘭縣', '頭城鎮', '', ''))],
 [('冬山車站', ('',

In [16]:
temp = open("site_temp.txt","wt",encoding='utf-8')
n = temp.write(str(data))
temp.close()

In [42]:
data2[0]

['清水斷崖', '', '', '花蓮縣', '秀林鄉', '', '', '', '']

In [51]:
data1 = str(data)

data1 = re.sub(r"\(|\)","",data1)

data2 = eval(data1)

data3 = [list(filter(None, sub_list)) for sub_list in data2]
# for sub_list in data2:
#     for x in sub_list:
#         x
data3[0]

['清水斷崖', '花蓮縣', '秀林鄉']

In [66]:
df = pd.DataFrame(data3,columns=['site','city','town'])
df = df.mask(df.astype(object).eq('None')).dropna()
df

,site,city,town
0,清水斷崖,花蓮縣,秀林鄉
1,太平山國家森林遊樂區,宜蘭縣,大同鄉
2,桃園機場捷運沿線景點,桃園市,蘆竹區
6,基隆正濱漁港,基隆市,中正區
7,阿根納造船廠遺址,基隆市,中正區
...,...,...,...
356,鳳山懷舊之旅,高雄市,鳳山區
357,東港大鵬灣,屏東縣,東港鎮
358,屏東東港~東隆宮,屏東縣,東港鎮
359,萬巒鄉情&豬腳美食,屏東縣,萬巒鄉


In [68]:
df.to_csv("taiwansite.csv",encoding="utf-8",index=False)